In [2]:
# all the functions are add-on to yfinance to have a more comprehensive dataset.
import yfinance as yf
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
options = webdriver.SafariOptions()
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')

In [ ]:
#print(nvda.option_chain('2025-02-21'))
#print(nvda.history(period="max"))
def convert_strdata(data):
    for item in data:
        for idx, number in enumerate(item):
            number = number.strip()
            if 'T' in number:
                value_num = number.rstrip(number[-1])
                item[idx] = float(value_num)*1000000000000
            elif 'B' in number:
                value_num = number.rstrip(number[-1])
                item[idx] = float(value_num)*1000000000
            elif 'M' in number:
                value_num = number.rstrip(number[-1])
                item[idx] = float(value_num)*1000000
            elif 'K' in number:
                print(number)
                value_num = number.rstrip(number[-1])
                item[idx] = float(value_num)*1000
            else:
                item[idx] = float(number)
    return data

In [ ]:
user_input = input("Please input the ticker(s) that you want to do analysis of, with white spaces in between: ")
user_input = user_input.split(sep=' ')

In [88]:
browser = webdriver.Safari(keep_alive=True, options=options)
browser.maximize_window()
browser.get('https://finance.yahoo.com/quote/'+user_input[0].upper()+'/news/')

'''
quote = browser.find_element(By.ID, 'ybar-sbq')
quote.clear()
quote.send_keys(user_input[0].upper())
send_quote_btn = browser.find_element(By.ID, 'ybar-search')
send_quote_btn.send_keys(Keys.RETURN)
time.sleep(1)
'''
url = f'/quote/{user_input[0].upper()}/'
# assert 'NVDA' in browser.page_source
time.sleep(1)
def to_stat_page():
    btn = browser.find_element(By.LINK_TEXT, 'Statistics')
    btn.send_keys(Keys.RETURN)
    time.sleep(2)

def to_news_page():
    btn = browser.find_element(By.XPATH, '//a[@href="'+url+'news/"]')
    btn.send_keys(Keys.RETURN)
    time.sleep(2)    

def to_summary_page():
    btn = browser.find_element(By.LINK_TEXT, 'Summary')
    btn.send_keys(Keys.RETURN)
    time.sleep(2)

def to_analysis_page():
    btn = browser.find_element(By.LINK_TEXT, 'Analysis')
    btn.send_keys(Keys.RETURN)
    time.sleep(2)

to_stat_page()


In [ ]:
def scroll_down(self):
    i = 0
    # Get scroll height.
    last_height = self.execute_script("return document.body.scrollHeight")
    while True:
        i += 1
        if i == 4:
            break
        else:
            # Scroll down to the bottom.
            self.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            # Calculate new scroll height and compare with last scroll height.
            new_height = self.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

In [ ]:
def get_valuation_table() -> pd.DataFrame:
    to_stat_page()
    metadata = []
    metaheader = []
    for table in browser.find_elements(By.XPATH, "//table[@class='table svelte-104jbnt']//tr"):
        data = [item.text for item in table.find_elements(By.XPATH, ".//*[self::td]")]
        header = [item.text for item in table.find_elements(By.XPATH, ".//*[self::th]")]
        metadata.append(data)
        if header != []:
            metaheader.append(header)
    metadata = [data for data in metadata if data != []]
    row = list(zip(*metadata))[0]
    for item in metadata:
        del item[0]

    metaheader = list(filter(None, metaheader[0]))
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    valuation_table = pd.DataFrame(convert_strdata(metadata),index=row, columns=metaheader)
    return valuation_table
df = get_valuation_table()
print(df.to_html)



In [ ]:
def get_financial_ratio() -> pd.DataFrame:
    to_stat_page()
    metadata = []
    metaheader = []
    fin_sum_section = browser.find_elements(By.XPATH, "//section[@class='svelte-14j5zka']")
    for table in fin_sum_section[0].find_elements(By.XPATH, ".//*[self::tr]"):
        *header, data = table.text.split()
        header = ' '.join(header)
        metadata.append(data)
        metaheader.append(header)
    finance_summary = pd.Series(dict((name, value) for name, value in list(zip(metaheader, metadata)))).to_frame()
    return finance_summary
        
get_financial_ratio()

In [ ]:
def get_share_stat() -> pd.DataFrame:
    #to_stat_page()
    metadata = []
    metaheader = []
    fin_sum_section = browser.find_elements(By.XPATH, "//section[@class='svelte-14j5zka']")
    for table in fin_sum_section[1].find_elements(By.XPATH, ".//*[self::tr]"):
        *header, data = table.text.split()
        header = ' '.join(header)
        try:
            int(header[-1])
            header = header[:-1]
            header = header.strip()
        except:
            continue
        metadata.append(data)
        metaheader.append(header)
    share_stat = dict((name, value) for name, value in list(zip(metaheader, metadata)))
    share_stat = pd.Series(share_stat).to_frame()
    return share_stat

get_share_stat()

In [ ]:
news_website = ['Bloomberg', 'Reuters', 'Investopedia', 'The Telegraph', 'The Wall Street Journal',
                'Barrons.com', 'Fortune', 'Retail Dive', 'WSJ', 'Yahoo Finance']

def get_news(): # news with only a few trusted source, and a separate column for official press release
    #to_news_page()
    btn = browser.find_element(By.ID, "tab-latest-news")
    btn.send_keys(Keys.RETURN)
    scroll_down(browser)
    test = browser.find_elements(By.XPATH, "//section[@data-testid='storyitem']")
    metaheader = []
    metadata = []
    metalink = []
    for idx in range(len(test)):
        header = test[idx].find_element(By.XPATH, ".//*[self::h3]").text
        src = [i.strip() for i in test[idx].find_elements(By.XPATH, ".//*[self::div]")[-1].text.split(sep='•')]
        link = test[idx].find_element(By.XPATH, ".//*[self::a]").get_attribute('href')
        if src[0] in news_website:
            metaheader.append(header)
            metadata.append(src)
            metalink.append(link)
    return metaheader, metadata, metalink

def get_press_release():
    #to_news_page()
    btn = browser.find_element(By.ID, "tab-press-releases")
    btn.send_keys(Keys.RETURN)
    time.sleep(1)
    test = browser.find_elements(By.XPATH, "//section[@data-testid='storyitem']")
    metaheader = []
    metadata = []
    metalink = []
    for idx in range(len(test)):
        header = test[idx].find_element(By.XPATH, ".//*[self::h3]").text
        src = [i.strip() for i in test[idx].find_elements(By.XPATH, ".//*[self::div]")[-1].text.split(sep='•')]
        link = test[idx].find_element(By.XPATH, ".//*[self::a]").get_attribute('href')
        metaheader.append(header)
        metadata.append(src)
        metalink.append(link)
    d = list(zip(metaheader, metadata, metalink))
    return d

def get_sec_filing(): # separate by 10-K, 10-Q, others
    to_news_page()
    element = browser.find_element(By.XPATH,value='//*[contains(text(), "All SEC Filings")]')                                                                                                            
    element.send_keys(Keys.RETURN)

d = get_press_release()
print(d)


In [ ]:
def get_competitors():
    to_summary_page()
    compare_to_section = browser.find_elements(By.XPATH, "//section[@data-testid='compare-to']//section")
    metadata = []
    column = ['Price', '% Change']
    for idx in range(len(compare_to_section)):
        element = compare_to_section[idx].find_elements(By.XPATH, ".//*[self::span]")
        data = [i.text for i in element]
        metadata.append(data[:3])
    row = [metadata[i][0] for i in range(len(metadata))]
    metadata = [metadata[i][1:] for i in range(len(metadata))]
    competitors = pd.DataFrame(metadata, index=row, columns=column)
    return competitors

In [ ]:
def get_perf_vs_mkt():
    to_news_page()
    perf_overview_section = browser.find_elements(By.XPATH, "//section[@data-testid='performance-overview-on-news']//table")
    column = [item.text for item in perf_overview_section[0].find_elements(By.XPATH, ".//*[self::th]")]
    column = column[1:]
    metadata = []
    for i in perf_overview_section[0].find_elements(By.XPATH, ".//*[self::tr]"):
        data = [item.text for item in i.find_elements(By.XPATH, ".//*[self::td]")]
        metadata.append(data)
    row = [metadata[i][0] for i in range(len(metadata))]
    metadata = [metadata[i][1:] for i in range(len(metadata))]
    perf_overview_table = pd.DataFrame(metadata, index=row, columns=column)
    return perf_overview_table

get_perf_vs_mkt()


In [86]:
def data_to_df(section):
    column = [item.text for item in section.find_elements(By.XPATH, ".//*[self::th]")]
    column = column[1:]
    metadata = []
    for i in section.find_elements(By.XPATH, ".//*[self::tr]"):
        data = [item.text for item in i.find_elements(By.XPATH, ".//*[self::td]")]
        metadata.append(data)
    metadata = [data for data in metadata if data != []]
    row = [metadata[i][0] for i in range(len(metadata))]
    metadata = [metadata[i][1:] for i in range(len(metadata))]
    df = pd.DataFrame(metadata, index=row, columns=column)
    return df
def get_estimates():
    df5 = get_rev_est()
    df1 = get_earnings_est()
    df2 = get_earnings_hist()
    df3 = get_growth_est()
    df4 = get_eps_trend()
    return df1, df2, df3, df4, df5
def get_rev_est():
    to_analysis_page()
    rev_est_section = browser.find_element(By.XPATH, "//section[@data-testid='revenueEstimate']//table")
    rev_est = data_to_df(rev_est_section)
    return rev_est

def get_earnings_est():
    #to_analysis_page()
    earnings_est_section = browser.find_element(By.XPATH, "//section[@data-testid='earningsEstimate']//table")
    earnings_est = data_to_df(earnings_est_section)
    return earnings_est

def get_earnings_hist():
    #to_analysis_page()
    earnings_hist_section = browser.find_element(By.XPATH, "//section[@data-testid='earningsHistory']//table")
    earnings_hist = data_to_df(earnings_hist_section)
    return earnings_hist

def get_growth_est():
    #to_analysis_page()
    growth_est_section = browser.find_element(By.XPATH, "//section[@data-testid='growthEstimate']//table")
    growth_est = data_to_df(growth_est_section)
    return growth_est

def get_eps_trend():
    #to_analysis_page()
    eps_section = browser.find_element(By.XPATH, "//section[@data-testid='epsTrend']//table")
    eps_trend = data_to_df(eps_section)
    return eps_trend

In [87]:
print(len(get_estimates()))

5


browser.refresh()

In [ ]:
btn = browser.find_element(By.ID, "tab-sec-filing")
btn.send_keys(Keys.ENTER)


In [ ]:
element = browser.find_element(By.XPATH,value='//*[contains(text(), "All SEC Filings")]')                                                                                                            
element.send_keys(Keys.RETURN)

In [ ]:
btn = browser.find_element(By.ID, "tab-sec-filing")
btn.send_keys(Keys.ENTER)
element = browser.find_element(By.XPATH,value='//*[contains(text(), "All SEC Filings")]')                                                                                                            
element.send_keys(Keys.RETURN)
time.sleep(1)
sec_filing = browser.find_elements(By.XPATH, "//div[@class='sec-story-container svelte-nhb3xj']")
metalink = []
metaheader = []
metadate = []
for idx in range(len(sec_filing)):
    sec_filing = browser.find_elements(By.XPATH, "//div[@class='sec-story-container svelte-nhb3xj']")
    #WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, ".//*[self::a]")))
    if sec_filing[idx].text[0] == '1':
        element = sec_filing[idx].find_element(By.XPATH, ".//*[self::a]")
        metaheader.append(sec_filing[idx].find_element(By.XPATH, ".//*[self::h3]").text)
        filing_date = sec_filing[idx].find_elements(By.XPATH, ".//div//*[self::div]")[1].text
        metadate.append(filing_date.strip())
        element.send_keys(Keys.RETURN)
        time.sleep(2)
        link = browser.find_element(By.XPATH, "//a[@class='Bgc($linkColor) Bgc($linkActiveColor):h Fz(s) Cur(p) Fw(500) Py(8px) Px(10px) C(white) Bd(0) Bdc($linkColor) Bdrs(18px) Miw(120px) Va(t) D(ib) Ta(c) Td(n)']")
        metalink.append(link.get_attribute('href'))
        browser.back()
        time.sleep(2)
sec_filings = list(zip(metaheader, metadate, metalink))
sec_filings

In [ ]:
browser.back()

In [ ]:
sec_filing = browser.find_elements(By.XPATH, "//div[@class='sec-story-container svelte-nhb3xj']")
metadata = []
for idx in range(len(sec_filing)):
    #WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, ".//*[self::a]")))
    sec_filing = browser.find_elements(By.XPATH, "//div[@class='sec-story-container svelte-nhb3xj']")
    try:
        int(sec_filing[idx].text[0])
        if int(sec_filing[idx].text[0]) == 1:
            element = sec_filing[idx].find_element(By.XPATH, ".//*[self::a]")
            element.send_keys(Keys.RETURN)
            time.sleep(2)
            data = browser.find_element(By.XPATH, "//a[@class='Bgc($linkColor) Bgc($linkActiveColor):h Fz(s) Cur(p) Fw(500) Py(8px) Px(10px) C(white) Bd(0) Bdc($linkColor) Bdrs(18px) Miw(120px) Va(t) D(ib) Ta(c) Td(n)']")
            metadata.append(data.get_attribute('href'))
            browser.back()
            time.sleep(2)
    except: continue
print(metadata)

In [ ]:
for i in titles:
    news_similarity = fuzz.ratio(i, article_title)              # check if the news titles are similar to the previous ones using the fuzz function fomr fuzzywuzzy
    print(f"{news_similarity}: {i}; {article_title}")
    if news_similarity > 50:
        same_news = True
        break
    else:
        same_news = False

In [ ]:
from cs50 import SQL
db = SQL("sqlite:///history.db")

In [ ]:
test = db.execute("SELECT username FROM users WHERE id = '1'")
test

In [19]:
stock_data = yf.download("RTX", start="2022-07-07", end="2024-7-7", group_by="ticker")
import plotly.graph_objs as go
fig = go.Figure(data=[go.Candlestick(x=stock_data.index, open=stock_data['Open'], high=stock_data['High'], low=stock_data['Low'], close=stock_data['Close'], name='Stock Data')])
fig.update_layout(yaxis_title='Stock Price (USD)', xaxis_title='Date')
fig.show()

[*********************100%%**********************]  1 of 1 completed


In [179]:
data = yf.download("NVDA TSLA", start="2220-2-29", end="2024-6-21", group_by="ticker")
data

[*********************100%%**********************]  2 of 2 completed

2 Failed downloads:
['TSLA', 'NVDA']: YFChartError('%ticker%: Invalid input - start date cannot be after end date. startDate = 7894299600, endDate = 1718942400')


Empty DataFrame
Columns: [(TSLA, Open), (TSLA, High), (TSLA, Low), (TSLA, Close), (TSLA, Adj Close), (TSLA, Volume), (NVDA, Open), (NVDA, High), (NVDA, Low), (NVDA, Close), (NVDA, Adj Close), (NVDA, Volume)]
Index: []

In [8]:
browser = webdriver.Safari(keep_alive=False, options=options)
browser.maximize_window()
browser.get("https://stockanalysis.com/stocks/")
metadata = []
table = browser.find_element(By.XPATH, "//div[@class='table-wrap svelte-172ru7t']/table")
for page in range(12):
    for row in table.find_elements(By.XPATH, './/*[self::tr]'):
        data = [item.text for item in row.find_elements(By.XPATH, ".//*[self::td]")]
        metadata.append(data)
    print(metadata)
    time.sleep(2)
    btn = browser.find_element(By.XPATH, "//button[@class='controls-btn xs:pl-1 xs:pr-1.5 bp:text-sm sm:pl-3 sm:pr-1']")
    btn.send_keys(Keys.RETURN)

print(metadata)

[[], ['A', 'Agilent Technologies, Inc.', 'Diagnostics & Research', '36.59B'], ['AA', 'Alcoa Corporation', 'Aluminum', '7.24B'], ['AACG', 'ATA Creativity Global', 'Education & Training Services', '26.33M'], ['AACI', 'Armada Acquisition Corp. I', 'Shell Companies', '81.25M'], ['AACT', 'Ares Acquisition Corporation II', 'Shell Companies', '667.81M'], ['AADI', 'Aadi Bioscience, Inc.', 'Biotechnology', '34.13M'], ['AAGR', 'African Agriculture Holdings Inc.', 'Farm Products', '9.64M'], ['AAL', 'American Airlines Group Inc.', 'Airlines', '7.20B'], ['AAMC', 'Altisource Asset Management Corporation', 'Mortgage Finance', '5.36M'], ['AAME', 'Atlantic American Corporation', 'Insurance - Life', '31.82M'], ['AAN', "The Aaron's Company, Inc.", 'Rental & Leasing Services', '314.59M'], ['AAOI', 'Applied Optoelectronics, Inc.', 'Communication Equipment', '330.33M'], ['AAON', 'AAON, Inc.', 'Building Products & Equipment', '6.79B'], ['AAP', 'Advance Auto Parts, Inc.', 'Specialty Retail', '3.45B'], ['AAPL'

In [9]:
metadata

[[],
 ['A', 'Agilent Technologies, Inc.', 'Diagnostics & Research', '36.59B'],
 ['AA', 'Alcoa Corporation', 'Aluminum', '7.24B'],
 ['AACG', 'ATA Creativity Global', 'Education & Training Services', '26.33M'],
 ['AACI', 'Armada Acquisition Corp. I', 'Shell Companies', '81.25M'],
 ['AACT', 'Ares Acquisition Corporation II', 'Shell Companies', '667.81M'],
 ['AADI', 'Aadi Bioscience, Inc.', 'Biotechnology', '34.13M'],
 ['AAGR', 'African Agriculture Holdings Inc.', 'Farm Products', '9.64M'],
 ['AAL', 'American Airlines Group Inc.', 'Airlines', '7.20B'],
 ['AAMC',
  'Altisource Asset Management Corporation',
  'Mortgage Finance',
  '5.36M'],
 ['AAME', 'Atlantic American Corporation', 'Insurance - Life', '31.82M'],
 ['AAN', "The Aaron's Company, Inc.", 'Rental & Leasing Services', '314.59M'],
 ['AAOI',
  'Applied Optoelectronics, Inc.',
  'Communication Equipment',
  '330.33M'],
 ['AAON', 'AAON, Inc.', 'Building Products & Equipment', '6.79B'],
 ['AAP', 'Advance Auto Parts, Inc.', 'Specialty 

In [45]:
'''
i = 0
for row in metadata:
    i+=1
    print(i)
    print(row[0])
    '''
ticker_list = [row[0] for row in metadata]
print(ticker_list)
if ['AAPL', 'NFLX', 'C'] in ticker_list:
    print('YES')

['A', 'AA', 'AACG', 'AACI', 'AACT', 'AADI', 'AAGR', 'AAL', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAT', 'AB', 'ABAT', 'ABBV', 'ABCB', 'ABCL', 'ABEO', 'ABEV', 'ABG', 'ABIO', 'ABL', 'ABLV', 'ABM', 'ABNB', 'ABOS', 'ABR', 'ABSI', 'ABT', 'ABTS', 'ABUS', 'ABVC', 'ABVE', 'ABVX', 'AC', 'ACA', 'ACAB', 'ACAC', 'ACAD', 'ACB', 'ACCD', 'ACCO', 'ACDC', 'ACEL', 'ACET', 'ACGL', 'ACHC', 'ACHL', 'ACHR', 'ACHV', 'ACI', 'ACIC', 'ACIU', 'ACIW', 'ACLS', 'ACLX', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACNT', 'ACON', 'ACR', 'ACRE', 'ACRS', 'ACRV', 'ACST', 'ACT', 'ACTG', 'ACU', 'ACVA', 'ACXP', 'ADAG', 'ADAP', 'ADBE', 'ADC', 'ADCT', 'ADD', 'ADEA', 'ADI', 'ADIL', 'ADM', 'ADMA', 'ADN', 'ADNT', 'ADP', 'ADPT', 'ADRT', 'ADSE', 'ADSK', 'ADT', 'ADTN', 'ADTX', 'ADUS', 'ADV', 'ADVM', 'ADXN', 'AE', 'AEAE', 'AEE', 'AEG', 'AEHL', 'AEHR', 'AEI', 'AEIS', 'AEM', 'AEMD', 'AENT', 'AEO', 'AEON', 'AEP', 'AER', 'AERT', 'AES', 'AESI', 'AEVA', 'AEYE', 'AEZS', 'AFAR', 'AFBI', 'AFCG', 'AFG', 'AFJK', 'AFL', 'AFMD', 'AFRI', 'AFRM

In [48]:
set(['AAPL', 'NFLX', 'C']).issubset(ticker_list)

True

In [52]:
import csv
with open('stocks.csv', 'w') as csv_file:
    text = csv.writer(csv_file, delimiter=',')
    text.writerow(ticker_list)

In [6]:
with open('stocks.csv', 'r') as stock_name_file:
    li = stock_name_file.read().split(sep=',')
li

['A',
 'AA',
 'AACG',
 'AACI',
 'AACT',
 'AADI',
 'AAGR',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AB',
 'ABAT',
 'ABBV',
 'ABCB',
 'ABCL',
 'ABEO',
 'ABEV',
 'ABG',
 'ABIO',
 'ABL',
 'ABLV',
 'ABM',
 'ABNB',
 'ABOS',
 'ABR',
 'ABSI',
 'ABT',
 'ABTS',
 'ABUS',
 'ABVC',
 'ABVE',
 'ABVX',
 'AC',
 'ACA',
 'ACAB',
 'ACAC',
 'ACAD',
 'ACB',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACEL',
 'ACET',
 'ACGL',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHV',
 'ACI',
 'ACIC',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACNT',
 'ACON',
 'ACR',
 'ACRE',
 'ACRS',
 'ACRV',
 'ACST',
 'ACT',
 'ACTG',
 'ACU',
 'ACVA',
 'ACXP',
 'ADAG',
 'ADAP',
 'ADBE',
 'ADC',
 'ADCT',
 'ADD',
 'ADEA',
 'ADI',
 'ADIL',
 'ADM',
 'ADMA',
 'ADN',
 'ADNT',
 'ADP',
 'ADPT',
 'ADRT',
 'ADSE',
 'ADSK',
 'ADT',
 'ADTN',
 'ADTX',
 'ADUS',
 'ADV',
 'ADVM',
 'ADXN',
 'AE',
 'AEAE',
 'AEE',
 'AEG',
 'AEHL',
 'AEHR',
 'AEI',
 'AEIS',
 'AEM',
 'AEMD',
 'AENT',
 'AEO',
 'AEON',
 'AEP',
 'AER',

In [10]:
for ticker in li:
    testing = yf.Ticker(ticker)
    print(ticker)
    testing.insider_transactions

A
AA
AACG


YFDataException: Failed to parse holders json data.